# DESeq2 with Salmon and STAR

[Resources] (http://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html)

In [ ]:
options(stringsAsFactors = FALSE)

In [ ]:
library(ggplot2)
library(data.table)
library(limma)
library(DESeq2)
library(RColorBrewer)
library(tximport)
library(readr)
library(tximportData)
library(tximeta)
library(GenomicFeatures)
library(apeglm)
library(gplots)

## Create a symbolic link to data

In [ ]:
system("ln -sfn ~/public/rnaseq/Day2_materials/* ~/module-3-rnaseq/Day2_materials/")

## Questions:
1) We know that Salmon is faster than STAR, but is it better?
2) How similar are Salmon and STAR in terms of differentially expressed genes?
3) How many replicates do we need?

**We have generated the quant.sh files from Salmon using the following commands:**
1) Generate a Salmon index (this takes about 8 minutes):

*/path_to/_where_you/_installed_salmon/bin/salmon index -t gencode.v44.transcripts.fa -i gencode_index --gencode*

2) Quant the samples:

*for i in *_R1.fastq.gz; do /path_to/_where_you/_installed_salmon/bin/salmon quant -i /path_to/_where_you/_output_your_index/gencode_index/ -l A -1 $i -2 ${i%_R1.fastq.gz}_R2.fastq.gz -p 13 --gcBias -o salmon_quant_output/${i%_R1.fastq.gz}_quant*

*done*

**How long did the quant take?**

Use a timer program in your script or if using a slurm system check by using (sacct -l -j <myid>). Alternatively, check the time stamp in the salmon_quant.log

First recorded time stamp: 2024-01-16 23:41:08.116


Last recorded time stamp: 2024-01-17 02:10:14.636


- took about two and a half hours for 20 samples or roughly 7.5 minutes per sample (run using 13 threads mem=96GB)

**What does a quant.sh file look like?**

Name	Length	EffectiveLength	TPM	NumReads

ENST00000456328.2	1657	1519.063	0.135470	15.498

ENST00000450305.2	632	393.000	0.000000	0.000

ENST00000488147.1	1351	1153.174	2.780315	241.453

ENST00000619216.1	68	68.000	0.000000	0.000

ENST00000473358.1	712	472.000	0.000000	0.000

ENST00000469289.1	535	297.000	0.000000	0.000

ENST00000607096.1	138	3.000	0.000000	0.000

ENST00000417324.1	1187	947.000	0.000000	0.000

Remember that these are pseudo-counts per transcript and we want counts per gene...

# Make a directory to store output

In [ ]:
system("mkdir ~/module-3-rnaseq/Day2_materials/output")

# Step 1 for Salmon pseudo-count data
Use tximport to import and combine the quant.sh files created for each sample

In [ ]:
#To use DESeq2 we need to first have a sample table that tells DESeq2 what samples we are interested in and an covariates we want to add to the GLM.
#First read in the sample table
samples <- read.table("./data/design_012424.csv", sep=',', header=TRUE)

In [ ]:
#Take a look at the samples table
samples

Check to make sure everything is there...

In [ ]:
files <- file.path("./salmon", samples$sample, "quant.sf")
all(file.exists(files))

Now we need to make the tx2gene file so that the transcripts can be associated with gene IDs for gene-level summarization. For this, we need to create a transcript to gene file that associates each transcript with its gene

In [ ]:
gtfPath <- file.path("./data/", "gencode.v44.annotation.gtf.gz")
txdb <- makeTxDbFromGFF(file=gtfPath)
k <- keys(txdb, keytype = "TXNAME")
tx2gene <- select(txdb, k, "GENEID", "TXNAME")

In [ ]:
names(files) <- paste0(samples$sample)
txi.salmon <- tximport(files, type = "salmon", tx2gene = tx2gene)
head(txi.salmon$counts)

Notice that we now no longer have ENST#'s but ENSG#'s. If we want to convert these to gene symbols we can do this using BioMart

In [ ]:
write.table(txi.salmon$counts, "./output/salmon_counts.csv", sep=",")

# Creating DESeq2 object

Read in counts file back in:

In [ ]:
counts<-read.csv("./output/salmon_counts.csv",header=TRUE,row.names=1)

Take a quick look at it

In [ ]:
head(counts)

Take a look at how big the count matrix is

In [ ]:
print(dim(counts))

**Pre-Filtering**

A lot of these genes have no counts and could therefore be filtered out. While it is not necessary to pre-filter low count genes before running the DESeq2 functions, there are two reasons which make pre-filtering useful: by removing rows in which there are very few reads, we reduce the memory size of the dds data object, and we increase the speed of count modeling within DESeq2. It can also improve visualizations, as features with no information for differential expression are not plotted in dispersion plots or MA-plots.

Here if we wanted to do some pre-filtering to keep only rows that have a count of at least 10 for a minimal number of samples we would do the below. The count of 10 is a reasonable choice for bulk RNA-seq. A recommendation for the minimal number of samples is to specify the smallest group size, e.g. here there are 3 treated samples. If there are not discrete groups, one can use the minimal number of samples where non-zero counts would be considered interesting. One can also omit this step entirely and just rely on the independent filtering procedures available in results(), either IHW or genefilter.

In [ ]:
#smallestGroupSize <- 3
#keep <- rowSums(counts(dds) >= 10) >= smallestGroupSize
#dds <- dds[keep,]

**Make DESeq2 object: Define counts matrix, groups, and design**
- countData: matrix of counts
- colData: dataframe with metadata for each sample
- design: name of column in colData we want to use as comparators

**Note about covariates, interactions, and multi-factor designs**
DESeq2 can analyze any possible experimental design that can be expressed with fixed effects terms (multiple factors, designs with interactions, designs with continuous variables, splines, and so on are all possible). By adding variables to the design, one can control for additional variation in the counts. For example, if the condition samples are balanced across experimental batches, by including the batch factor to the design, one can increase the sensitivity for finding differences due to condition. We can account for fetal_sex (or any other variable) and get a clearer picture of the differences attributable to the treatment (cell_type in our case). As cell_type is the variable of interest, we put it at the end of the formula. Thus the results function will by default pull the cell_type results unless contrast or name arguments are specified.

Most of the questions you have in the future will be having to do with this design formula. Please see the vignette and Google for more information:
https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#multi-factor-designs
https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#interactions

In [ ]:
DESeq2CDS = DESeqDataSetFromTximport(txi.salmon, colData=samples, design=~fetal_sex + cell_type)

This warning means that R is converting our text to categories.

# Run DESeq2

In [ ]:
#We can now run the differential expression analysis using this one command:
dds <- DESeq(DESeq2CDS, parallel = TRUE)

A very quick and dirty QC is the dispersion plot. This plot is the estimate for the expected dispersion value for genes of a given expression strength. Each black dot is a gene with an associated mean expression level and maximum likelihood estimation (MLE) of the dispersion. This is a good plot to examine to ensure your data is a good fit for the DESeq2 model. You expect your data to generally scatter around the curve, with the dispersion decreasing with increasing mean expression levels. If you see a cloud or different shapes, then you might want to explore your data more to see if you have contamination (mitochondrial, etc.) or outlier samples. 

In [ ]:
plotDispEsts(dds)

Let's take a quick look at the differential expression results:

In [ ]:
res <- results(dds)
res

Details about the comparison are printed to the console, directly above the results table. The text, condition treated vs untreated, tells you that the estimates are of the logarithmic fold change log2(EVT 1st/CTB 1st).

With no additional arguments to results, the log2 fold change and Wald test p value will be for the last variable in the design formula, and if this is a factor, the comparison will be **the last level of this variable over the reference level**. By default, R will choose a reference level for factors based on alphabetical order. Then, if you never tell the DESeq2 functions which level you want to compare against (e.g. which level represents the control group), the comparisons will be based on the alphabetical order of the levels. Normally you will want to explicitly tell results which comparison to make using the contrast argument, or you can explicitly set the factors levels (we won't be going over this). The order of the variables of the design do not matter so long as the user specifies the comparison to build a results table for, using the name or contrast arguments of results.

In [ ]:
res_test <- results(dds, contrast=c("cell_type","CTB_1st","EVT_1st"))
res_test

The results function contains a number of arguments to customize the results table which is generated. You can read about these arguments by looking up ?results. Note that the results function automatically performs independent filtering based on the mean of normalized counts for each gene, optimizing the number of genes which will have an adjusted p value below a given FDR cutoff, alpha. By default the argument alpha is set to 0.1. If the adjusted p value cutoff will be a value other than 0.1, alpha should be set to that value:

In [ ]:
#Because 0.05 is more generally expcepted we should do:
res05 <- results(dds, alpha=0.05,contrast=c("cell_type","EVT_1st","CTB_1st"))
summary(res05)

In [ ]:
#If we want to know how many genes in total were differentially expressed we could do
sum(res05$padj < 0.05, na.rm=TRUE)

# Normalization and Results Exporting

DESeq2 doesn’t use normalized counts, rather it uses the raw counts and models the normalization inside the Generalized Linear Model (GLM). It does perform normalization as we discussed in class and we can output the normalized data to use in other ways.

DESeq2 uses size factors to normalize data and if we would like to see these factors we can use the function [`estimateSizeFactors()`](https://rdrr.io/bioc/DESeq2/man/estimateSizeFactors.html), which "estimates the size factors using the "median ratio method" described by Equation 5 in Anders and Huber (2010)".

We can extract the normalized counts using counts() with the parameter normalized=TRUE.

In [ ]:
normalized_counts_salmon10 <- counts(dds, normalized=TRUE)
write.table(normalized_counts_salmon10, file="./output/DESeq2normalized_counts_salmon.txt", sep="\t", quote=F, col.names=NA)

We also often want to export our results so to do that we can do:

In [ ]:
#Sort our results file by adjusted p-value
res_salmon10 <- res05[order(res05$padj),]

#Then merge it with our normalized data table that we output above
resdata_salmon10 <- merge(as.data.frame(res_salmon10), as.data.frame(normalized_counts_salmon10), by="row.names", sort=FALSE)
write.table(resdata_salmon10,file="./output/DESeq2_resultsalpha0_05_salmon10.txt",sep="\t")

# QC and exploration of DESeq2 Results

**MA Plot**

In DESeq2, the function plotMA shows the log2 fold changes attributable to a given variable over the mean of normalized counts for all the samples in the DESeqDataSet. Points will be colored blue if the adjusted p value is less than 0.1. Points which fall out of the window are plotted as open triangles pointing either up or down.

In [ ]:
plotMA(res, ylim=c(-5,9))

It is more useful to visualize the MA-plot for the shrunken log2 fold changes, which remove the noise associated with log2 fold changes from low count genes without requiring arbitrary filtering thresholds.

In [ ]:
resultsNames(dds)

In [ ]:
#Log fold change shrinkage for visualization and ranking
resLFC <- lfcShrink(dds, coef="cell_type_EVT_1st_vs_CTB_1st",type="apeglm")

In [ ]:
#Now we can plot the MA with these values
plotMA(resLFC, ylim=c(-5,9))

## PCA Plot
Related to the distance matrix is the PCA plot, which shows the samples in the 2D plane spanned by their first two principal components. This type of plot is useful for visualizing the overall effect of experimental covariates and batch effects.

We need to first transform the data to better visualize it. To do this we can use either variance stabilizing transformation (vst) or regularized log transformation using `rlog()`. Variance stabilizing transformation which is roughly similar to putting the data on the log2 scale, while also dealing with the sampling variability of low counts. It uses the design formula to calculate the within-group variability (if blind=FALSE) or the across-all-samples variability (if blind=TRUE). It does not use the design to remove variation in the data. It therefore does not remove variation that can be associated with batch or other covariates (nor does DESeq2 have a way to specify which covariates are nuisance and which are of interest).

It is possible to visualize the transformed data with batch variation removed, using the removeBatchEffect function from limma. This simply removes any shifts in the log2-scale expression data that can be explained by batch.

In [ ]:
vsd <- vst(dds, blind=FALSE)

In [ ]:
#plot PCA
pcaData <- plotPCA(vsd, intgroup = c("cell_type","fetal_sex"), returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=cell_type, shape=fetal_sex)) +
  geom_point(size=3) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
  coord_fixed()

## Heatmap of sample-to-sample distances

A heatmap of this distance matrix gives us an overview over similarities and dissimilarities between samples.

The dist() function "computes and returns the distance matrix computed by using the specified distance measure to compute the distances between the rows of a data matrix".


In [ ]:
#visualize sample distances
sampleDists <- dist(t(assay(vsd)))
sampleDistMatrix <- as.matrix(sampleDists)
head(sampleDistMatrix)

In [ ]:
#Plot using heatmap2
colours = colorRampPalette(rev(brewer.pal(9, "Blues")))(255)
heatmap.2(sampleDistMatrix, trace="none", col=colours)

## Volcano plot

The volcano plot enables to simultaneously capture the effect size and significance (ordinate) of each tested gene.

In [ ]:
#volcano plot
#reset par
par(mfrow=c(1,1))
# Make a basic volcano plot
with(res, plot(log2FoldChange, -log10(pvalue), pch=20, main="Volcano plot", xlim=c(-10,10),ylim=c(0,200)))

# Add colored points: blue if padj<0.01, red if log2FC>1 and padj<0.05)
with(subset(res, padj<.01 ), points(log2FoldChange, -log10(pvalue), pch=20, col="blue"))
with(subset(res, padj<.01 & abs(log2FoldChange)>2), points(log2FoldChange, -log10(pvalue), pch=20, col="red"))

# How many samples do you need?

**First Re-run with the salmon output**

In [ ]:
#I need to read in the design meta-data first as well as the count matix that we exported from salmon
design8 = read.csv("./data/design_012424_8samples.csv", header=TRUE)
files <- file.path("./salmon", design8$sample, "quant.sf")
gtfPath <- file.path("./data/", "gencode.v44.annotation.gtf.gz")
txdb <- makeTxDbFromGFF(file=gtfPath)
k <- keys(txdb, keytype = "TXNAME")
tx2gene <- select(txdb, k, "GENEID", "TXNAME")
names(files) <- paste0(design8$sample)
txi.salmon <- tximport(files, type = "salmon", tx2gene = tx2gene)

**Now re-run DESeq2**

In [ ]:
DESeq2CDS_s8 = DESeqDataSetFromTximport(txi.salmon, colData=design8, design=~fetal_sex + cell_type)
#Run DESeq2
dds_s8 <- DESeq(DESeq2CDS_s8, parallel = TRUE)
res_s8 <- results(dds_s8, alpha=0.05,contrast=c("cell_type","EVT_1st","CTB_1st"))
#How many genes in total are differentially expressed
sum(res_s8$padj < 0.05, na.rm=TRUE)

**Now repeat for 6 samples per group, then 4, and finally 3**

In [ ]:
#I need to read in the design meta-data first as well as the count matix that we exported from salmon
design6 = read.csv("./data/design_012424_6samples.csv", header=TRUE)
files <- file.path("./salmon", design6$sample, "quant.sf")
gtfPath <- file.path("./data", "gencode.v44.annotation.gtf.gz")
txdb <- makeTxDbFromGFF(file=gtfPath)
k <- keys(txdb, keytype = "TXNAME")
tx2gene <- select(txdb, k, "GENEID", "TXNAME")
names(files) <- paste0(design6$sample)
txi.salmon <- tximport(files, type = "salmon", tx2gene = tx2gene)
DESeq2CDS_s6 = DESeqDataSetFromTximport(txi.salmon, colData=design6, design=~fetal_sex + cell_type)
#Run DESeq2
dds_s6 <- DESeq(DESeq2CDS_s6, parallel = TRUE)
res_s6 <- results(dds_s6, alpha=0.05,contrast=c("cell_type","EVT_1st","CTB_1st"))
#How many genes in total are differentially expressed
sum(res_s6$padj < 0.05, na.rm=TRUE)

In [ ]:
#I need to read in the design meta-data first as well as the count matix that we exported from salmon
design4 = read.csv("./data/design_012424_4samples.csv", header=TRUE)
files <- file.path("./salmon", design4$sample, "quant.sf")
gtfPath <- file.path("./data", "gencode.v44.annotation.gtf.gz")
txdb <- makeTxDbFromGFF(file=gtfPath)
k <- keys(txdb, keytype = "TXNAME")
tx2gene <- select(txdb, k, "GENEID", "TXNAME")
names(files) <- paste0(design4$sample)
txi.salmon <- tximport(files, type = "salmon", tx2gene = tx2gene)
DESeq2CDS_s4 = DESeqDataSetFromTximport(txi.salmon, colData=design4, design=~fetal_sex + cell_type)
#Run DESeq2
dds_s4 <- DESeq(DESeq2CDS_s4, parallel = TRUE)
res_s4 <- results(dds_s4, alpha=0.05,contrast=c("cell_type","EVT_1st","CTB_1st"))
#How many genes in total are differentially expressed
sum(res_s4$padj < 0.05, na.rm=TRUE)

In [ ]:
#I need to read in the design meta-data first as well as the count matix that we exported from salmon
design3 = read.csv("./data/design_012424_3samples.csv", header=TRUE)
files <- file.path("./salmon", design3$sample, "quant.sf")
gtfPath <- file.path("./data", "gencode.v44.annotation.gtf.gz")
txdb <- makeTxDbFromGFF(file=gtfPath)
k <- keys(txdb, keytype = "TXNAME")
tx2gene <- select(txdb, k, "GENEID", "TXNAME")
names(files) <- paste0(design3$sample)
txi.salmon <- tximport(files, type = "salmon", tx2gene = tx2gene)
DESeq2CDS_s3 = DESeqDataSetFromTximport(txi.salmon, colData=design3, design=~fetal_sex + cell_type)
#Run DESeq2
dds_s3 <- DESeq(DESeq2CDS_s3, parallel = TRUE)
res_s3 <- results(dds_s3, alpha=0.05,contrast=c("cell_type","EVT_1st","CTB_1st"))
#How many genes in total are differentially expressed
sum(res_s3$padj < 0.05, na.rm=TRUE)

## **How does this compare to mapping with STAR?**

In [ ]:
design = read.csv("./data/design_012424.csv", header=TRUE, row.names=1)
star = read.csv("./data/CMM262_01232024_counts.csv", header=TRUE, row.names=1)
DESeq2CDS_star = DESeqDataSetFromMatrix(countData=star, colData=design, design=~fetal_sex + cell_type)
dds_star <- DESeq(DESeq2CDS_star, parallel = TRUE)
res_star <- results(dds_star, alpha=0.05,contrast=c("cell_type","EVT_1st","CTB_1st"))
#How many genes in total are differentially expressed
sum(res_star$padj < 0.05, na.rm=TRUE)

Now look at 8, 6, 4, and 3 samples:

In [ ]:
design_star8 = read.csv("./data/design_012424_8samples.csv", header=TRUE, row.names=1)
star_8 = read.csv("./data/CMM262_01232024_counts_8.csv", header=TRUE, row.names=1)
DESeq2CDS_star8 = DESeqDataSetFromMatrix(countData=star_8, colData=design_star8, design=~fetal_sex + cell_type)
dds_star8 <- DESeq(DESeq2CDS_star8, parallel = TRUE)
res_star8 <- results(dds_star8, alpha=0.05,contrast=c("cell_type","EVT_1st","CTB_1st"))
#How many genes in total are differentially expressed
sum(res_star8$padj < 0.05, na.rm=TRUE)

In [ ]:
design_star6 = read.csv("./data/design_012424_6samples.csv", header=TRUE, row.names=1)
star_6 = read.csv("./data/CMM262_01232024_counts_6.csv", header=TRUE, row.names=1)
DESeq2CDS_star6 = DESeqDataSetFromMatrix(countData=star_6, colData=design_star6, design=~fetal_sex + cell_type)
dds_star6 <- DESeq(DESeq2CDS_star6, parallel = TRUE)
res_star6 <- results(dds_star6, alpha=0.05,contrast=c("cell_type","EVT_1st","CTB_1st"))
#How many genes in total are differentially expressed
sum(res_star6$padj < 0.05, na.rm=TRUE)

In [ ]:
design_star4 = read.csv("./data/design_012424_4samples.csv", header=TRUE, row.names=1)
star_4 = read.csv("./data/CMM262_01232024_counts_4.csv", header=TRUE, row.names=1)
DESeq2CDS_star4 = DESeqDataSetFromMatrix(countData=star_4, colData=design_star4, design=~fetal_sex + cell_type)
dds_star4 <- DESeq(DESeq2CDS_star4, parallel = TRUE)
res_star4 <- results(dds_star4, alpha=0.05,contrast=c("cell_type","EVT_1st","CTB_1st"))
#How many genes in total are differentially expressed
sum(res_star4$padj < 0.05, na.rm=TRUE)

In [ ]:
design_star3 = read.csv("./data/design_012424_3samples.csv", header=TRUE, row.names=1)
star_3 = read.csv("./data/CMM262_01232024_counts_3.csv", header=TRUE, row.names=1)
DESeq2CDS_star3 = DESeqDataSetFromMatrix(countData=star_3, colData=design_star3, design=~fetal_sex + cell_type)
dds_star3 <- DESeq(DESeq2CDS_star3, parallel = TRUE)
res_star3 <- results(dds_star3, alpha=0.05,contrast=c("cell_type","EVT_1st","CTB_1st"))
#How many genes in total are differential
sum(res_star3$padj < 0.05, na.rm=TRUE)

In [ ]:
star_salmon_degs <- read.csv("./data/DEGs_salmon_star.csv", header=TRUE)
star_salmon_df <- data.frame(star_salmon_degs)
star_salmon_df